In [41]:
import keras
from keras.layers import Activation, Dense, Input, Flatten, Dropout, Lambda, Softmax
from keras.models import Model
from keras import regularizers
from keras import backend as K
import tensorflow as tf

import numpy as np
from scipy import sparse

In [3]:
data = sparse.load_npz("raw.npz")

In [4]:
n_playlists = data.shape[0]
n_songs = data.shape[1]

n_test = int(0.1 * n_playlists)
n_val = int(0.1 * n_playlists)

playlist_ids = np.random.permutation(n_playlists)

test_ids = playlist_ids[:n_test]
val_ids = playlist_ids[n_test:n_test+n_val]
train_ids = playlist_ids[n_test+n_val:]

train_data = data.tocsr()[train_ids]
val_data = data.tocsr()[val_ids]
test_data = data.tocsr()[test_ids]

In [42]:
K.clear_session()

h = 200

inputs = Input(shape=(n_songs,), name='dae_input')
inputs_normed = Lambda(lambda  x: tf.nn.l2_normalize(x, dim=1))(inputs)
x = Dropout(0.5)(inputs_normed)
latent = Dense(h, activation='tanh', name='latent_vector', kernel_regularizer=regularizers.l2(0.01))(x)

encoder = Model(inputs, latent, name='encoder')
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dae_input (InputLayer)       (None, 69678)             0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 69678)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 69678)             0         
_________________________________________________________________
latent_vector (Dense)        (None, 200)               13935800  
Total params: 13,935,800
Trainable params: 13,935,800
Non-trainable params: 0
_________________________________________________________________


In [43]:
latent_inputs = Input(shape=(h,), name='dae_latent')

logits = Dense(n_songs, name='dae_output', kernel_regularizer=regularizers.l2(0.01))(latent_inputs)
probs = Softmax(name='dae_probs')(logits) 
decoder = Model(latent_inputs, probs, name='decoder')
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dae_latent (InputLayer)      (None, 200)               0         
_________________________________________________________________
dae_output (Dense)           (None, 69678)             14005278  
_________________________________________________________________
dae_probs (Softmax)          (None, 69678)             0         
Total params: 14,005,278
Trainable params: 14,005,278
Non-trainable params: 0
_________________________________________________________________


In [47]:
dae = Model(inputs=inputs, outputs=decoder(encoder(inputs)), name='autoencoder')
dae.summary()

dae.compile(loss='categorical_crossentropy', optimizer='adam')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dae_input (InputLayer)       (None, 69678)             0         
_________________________________________________________________
encoder (Model)              (None, 200)               13935800  
_________________________________________________________________
decoder (Model)              (None, 69678)             14005278  
Total params: 27,941,078
Trainable params: 27,941,078
Non-trainable params: 0
_________________________________________________________________


In [48]:
dae.fit(train_data,
    train_data,
    validation_data=(val_data, val_data),
    epochs=30,
    batch_size=500)

Train on 314992 samples, validate on 39374 samples
Epoch 1/30
 49000/314992 [===>..........................] - ETA: 54:21 - loss: 1045.7675

KeyboardInterrupt: 